In [1]:
import heapq


#  ENVIRONMENT GENERATION
def generate_environment(rows=5, cols=6):
    # Create a 2D grid filled with "Clean"
    room = [["Clean" for _ in range(cols)] for _ in range(rows)]

    # Predefined dirty positions
    dirty_positions = [(0, 1), (0, 4), (1, 1), (2, 3), (3, 0), (4, 2)]

    # Mark dirty cells
    for r, c in dirty_positions:
        room[r][c] = "Dirty"

    return room  # Return the room


#  HEURISTIC FUNCTION
def heuristic(a, b):
    # Manhattan Distance between two points
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


#  A* SEARCH ALGORITHM
def a_star(room, start, goal):
    rows, cols = len(room), len(room[0])  # Grid size

    open_list = []  # Priority queue
    heapq.heappush(open_list, (0, start))  # Push start node

    came_from = {}  # To reconstruct the path
    g_cost = {start: 0}  # Cost from start to each node

    while open_list:
        _, current = heapq.heappop(open_list)  # Get node with lowest cost

        # If goal is reached
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]  # Return reversed path

        # Explore neighbors (up, down, left, right)
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            neighbor = (current[0] + dx, current[1] + dy)

            # Check if neighbor is inside grid
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                new_g = g_cost[current] + 1  # Movement cost

                # Check if this path is better
                if neighbor not in g_cost or new_g < g_cost[neighbor]:
                    g_cost[neighbor] = new_g
                    f_cost = new_g + heuristic(neighbor, goal)  # f = g + h
                    heapq.heappush(open_list, (f_cost, neighbor))
                    came_from[neighbor] = current

    return None


#  VACUUM AGENT
def vacuum_agent(room, start=(0, 0)):
    current_pos = start  # Initial vacuum position
    total_path = []  # Store full movement path

    # Collect all dirty cells
    dirty_cells = [(i, j) for i in range(len(room))
                   for j in range(len(room[0]))
                   if room[i][j] == "Dirty"]

    # Loop until all dirty cells are cleaned
    while dirty_cells:
        # Choose nearest dirty cell
        target = min(dirty_cells, key=lambda x: heuristic(current_pos, x))

        # Find path using A*
        path = a_star(room, current_pos, target)

        if path:
            total_path.extend(path[1:])  # Add movement steps
            current_pos = target  # Move vacuum
            room[target[0]][target[1]] = "Clean"  # Clean cell
            dirty_cells.remove(target)  # Remove from list

    return total_path  # Return complete path


#  MAIN (RUN & OUTPUT)
room = generate_environment()  # Create room
path = vacuum_agent(room)  # Run vacuum agent

print("Vacuum Movement Path:")
print(path)  # Print movement path

print("\nFinal Room State:")
for row in room:
    print(row)  # Print room after cleaning
